In [1]:
#ch21_SparkML.ipynb, Spark ML Pipeline多元分類

In [2]:
sc.master

u'local[*]'

In [ ]:
#step1. 資料準備

In [1]:
rawData=sc.textFile("covtype.data")
lines=rawData.map(lambda x:x.split(","))
lines.count()

581012

In [3]:
fieldnum = len(lines.first()) #算欄位數
fieldnum

55

In [ ]:
#step2. 建立DataFrame
# pyspark.sql.types --> List of data types available.
#StructType is a built-in data type in Spark SQL to represent a collection of StructFields that together define a schema or its part.

In [4]:
#step2.1 匯入 StringType,StructField,StructType module
from pyspark.sql.types import  StringType,StructField,StructType

In [ ]:
#step2.2 建立DataFrame

In [5]:
#step2.2.1 建立欄位list fields
fields = [StructField("f"+str(i), StringType(), True)  #串列生成式
               for i in range(fieldnum )]
#fro i in range(fieldnum):       i--> 0~54
#以StructField() 建立欄位, 'f0','f1','f2',...'f54', 資料型別為 String
#結果存放在 fields --> list of string

In [6]:
fields #檢視

[StructField(f0,StringType,true),
 StructField(f1,StringType,true),
 StructField(f2,StringType,true),
 StructField(f3,StringType,true),
 StructField(f4,StringType,true),
 StructField(f5,StringType,true),
 StructField(f6,StringType,true),
 StructField(f7,StringType,true),
 StructField(f8,StringType,true),
 StructField(f9,StringType,true),
 StructField(f10,StringType,true),
 StructField(f11,StringType,true),
 StructField(f12,StringType,true),
 StructField(f13,StringType,true),
 StructField(f14,StringType,true),
 StructField(f15,StringType,true),
 StructField(f16,StringType,true),
 StructField(f17,StringType,true),
 StructField(f18,StringType,true),
 StructField(f19,StringType,true),
 StructField(f20,StringType,true),
 StructField(f21,StringType,true),
 StructField(f22,StringType,true),
 StructField(f23,StringType,true),
 StructField(f24,StringType,true),
 StructField(f25,StringType,true),
 StructField(f26,StringType,true),
 StructField(f27,StringType,true),
 StructField(f28,StringType,tr

In [7]:
#step2.2.2 建立schema
schema = StructType(fields)

In [8]:
#step2.2.3 檢視schema
schema

StructType(List(StructField(f0,StringType,true),StructField(f1,StringType,true),StructField(f2,StringType,true),StructField(f3,StringType,true),StructField(f4,StringType,true),StructField(f5,StringType,true),StructField(f6,StringType,true),StructField(f7,StringType,true),StructField(f8,StringType,true),StructField(f9,StringType,true),StructField(f10,StringType,true),StructField(f11,StringType,true),StructField(f12,StringType,true),StructField(f13,StringType,true),StructField(f14,StringType,true),StructField(f15,StringType,true),StructField(f16,StringType,true),StructField(f17,StringType,true),StructField(f18,StringType,true),StructField(f19,StringType,true),StructField(f20,StringType,true),StructField(f21,StringType,true),StructField(f22,StringType,true),StructField(f23,StringType,true),StructField(f24,StringType,true),StructField(f25,StringType,true),StructField(f26,StringType,true),StructField(f27,StringType,true),StructField(f28,StringType,true),StructField(f29,StringType,true),Stru

In [9]:
#step2.3 以spark.createDataFrame(RDD,schema) 建立spark DataFrame 'covtype_df'
covtype_df = spark.createDataFrame(lines, schema)

In [10]:
#檢視covtype_df 的欄位
print covtype_df.columns

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54']


In [11]:
#檢視covtype_df 的schema, printSchema()
print covtype_df.printSchema()

root
 |-- f0: string (nullable = true)
 |-- f1: string (nullable = true)
 |-- f2: string (nullable = true)
 |-- f3: string (nullable = true)
 |-- f4: string (nullable = true)
 |-- f5: string (nullable = true)
 |-- f6: string (nullable = true)
 |-- f7: string (nullable = true)
 |-- f8: string (nullable = true)
 |-- f9: string (nullable = true)
 |-- f10: string (nullable = true)
 |-- f11: string (nullable = true)
 |-- f12: string (nullable = true)
 |-- f13: string (nullable = true)
 |-- f14: string (nullable = true)
 |-- f15: string (nullable = true)
 |-- f16: string (nullable = true)
 |-- f17: string (nullable = true)
 |-- f18: string (nullable = true)
 |-- f19: string (nullable = true)
 |-- f20: string (nullable = true)
 |-- f21: string (nullable = true)
 |-- f22: string (nullable = true)
 |-- f23: string (nullable = true)
 |-- f24: string (nullable = true)
 |-- f25: string (nullable = true)
 |-- f26: string (nullable = true)
 |-- f27: string (nullable = true)
 |-- f28: string (nullabl

In [12]:
#step2.4 將欄位資料型別由string 轉成 double
from pyspark.sql.functions import col  #import col module, 用來存取欄位
covtype_df= covtype_df.select([ col(column).cast("double").alias(column) 
                                          for column in covtype_df.columns])

In [13]:
#再次檢視schema, string-> double
covtype_df.printSchema()

root
 |-- f0: double (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- f3: double (nullable = true)
 |-- f4: double (nullable = true)
 |-- f5: double (nullable = true)
 |-- f6: double (nullable = true)
 |-- f7: double (nullable = true)
 |-- f8: double (nullable = true)
 |-- f9: double (nullable = true)
 |-- f10: double (nullable = true)
 |-- f11: double (nullable = true)
 |-- f12: double (nullable = true)
 |-- f13: double (nullable = true)
 |-- f14: double (nullable = true)
 |-- f15: double (nullable = true)
 |-- f16: double (nullable = true)
 |-- f17: double (nullable = true)
 |-- f18: double (nullable = true)
 |-- f19: double (nullable = true)
 |-- f20: double (nullable = true)
 |-- f21: double (nullable = true)
 |-- f22: double (nullable = true)
 |-- f23: double (nullable = true)
 |-- f24: double (nullable = true)
 |-- f25: double (nullable = true)
 |-- f26: double (nullable = true)
 |-- f27: double (nullable = true)
 |-- f28: double (nullabl

In [ ]:
#**********************************************
#上述資料轉換,可用pandas 套件處理
import pandas as pd 
pd_df=pd.read_csv("covtype.data") 
#建立欄位字串串列 fields
fields=["f"+str(i) for i in range(55)] 
#更改column name
pd_df.columns=fields
#將int 改為 float
pd_df=pd_df.applymap(lambda x:float(x))
#sqlContext.createDataFrame()
covtype_df=sqlContext.createDataFrame(pd_df)
#*********************************************

In [14]:
#step2.5 建立feature column list
#              從DataFrame covtype_df 擷取 [:54]
featuresCols=covtype_df.columns[:54]
print(featuresCols)

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53']


In [15]:
#step2.6 建立label column, 並調整label 值
#step2.6.1 以DataFrame.withColumn(), 建立一個新欄位 "label",其值為原"f54"欄位值減一
#                 這是因為在DecisionTreeClassifier algorithm 中,規定 label 值需由0開始, 
#                 原始 label "f54"值,是 1,2,...,7, 調整後label值為 0,1,2,...,6
covtype_df=covtype_df.withColumn("label", covtype_df["f54"] -1)
#step2.6.2 以DataFrame.drop() 將原先的欄位 "f54" drop 
covtype_df=covtype_df.drop("f54")

In [16]:
#step2.6.3 以DataFrame.show() 顯示第一筆修改後的 資料
covtype_df.show(1)

+------+----+---+-----+---+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|    f0|  f1| f2|   f3| f4|   f5|   f6|   f7|   f8|    f9|f10|f11|f12|f13|f14|f15|f16|f17|f18|f19|f20|f21|f22|f23|f24|f25|f26|f27|f28|f29|f30|f31|f32|f33|f34|f35|f36|f37|f38|f39|f40|f41|f42|f43|f44|f45|f46|f47|f48|f49|f50|f51|f52|f53|label|
+------+----+---+-----+---+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|2596.0|51.0|3.0|258.0|0.0|510.0|221.0|232.0|148.0|6279.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|  4.0|
+------+----+---+-----+---+-----+---

In [17]:
covtype_df

DataFrame[f0: double, f1: double, f2: double, f3: double, f4: double, f5: double, f6: double, f7: double, f8: double, f9: double, f10: double, f11: double, f12: double, f13: double, f14: double, f15: double, f16: double, f17: double, f18: double, f19: double, f20: double, f21: double, f22: double, f23: double, f24: double, f25: double, f26: double, f27: double, f28: double, f29: double, f30: double, f31: double, f32: double, f33: double, f34: double, f35: double, f36: double, f37: double, f38: double, f39: double, f40: double, f41: double, f42: double, f43: double, f44: double, f45: double, f46: double, f47: double, f48: double, f49: double, f50: double, f51: double, f52: double, f53: double, label: double]

In [18]:
#step2.7 以DataFrame.randomSplit() 將資料分為 train_df與 test_df
#              比例為 7:3, 並以DataFrame.cache() 將資料cache 在MM
train_df, test_df = covtype_df.randomSplit([0.7, 0.3]) 
train_df.cache()
test_df.cache()

DataFrame[f0: double, f1: double, f2: double, f3: double, f4: double, f5: double, f6: double, f7: double, f8: double, f9: double, f10: double, f11: double, f12: double, f13: double, f14: double, f15: double, f16: double, f17: double, f18: double, f19: double, f20: double, f21: double, f22: double, f23: double, f24: double, f25: double, f26: double, f27: double, f28: double, f29: double, f30: double, f31: double, f32: double, f33: double, f34: double, f35: double, f36: double, f37: double, f38: double, f39: double, f40: double, f41: double, f42: double, f43: double, f44: double, f45: double, f46: double, f47: double, f48: double, f49: double, f50: double, f51: double, f52: double, f53: double, label: double]

In [ ]:
#step3. 建立 Machine Learning Pipeline

In [19]:
#step3.1 import Pipeline, VectorAssembler, DecisionTreeClassifier module
from pyspark.ml import Pipeline  
from pyspark.ml.feature import   VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

In [20]:
#step3.2 建立pipeline
#step3.2.1 建立 VectorAssembler 物件 'vectorAssembler'
#                將step2.5建立的featuresCols 整合成一個 ''features' Vector 
vectorAssembler = VectorAssembler(inputCols=featuresCols, 
                                                               outputCol="features")

In [23]:
#step3.2.2 建立DecisionTreeClassifier 物件 'dt'
#                maxDepth=5, maxBins=20, featuersCol 是 step3.2.1 產生的 Vector "features"
dt=DecisionTreeClassifier(labelCol="label",featuresCol="features",
                          maxDepth=5,maxBins=20)

In [24]:
#step3.2.3 建立 Pipeline物件 'dt_pipeline'
dt_pipeline=Pipeline(stages=[vectorAssembler, dt])

In [25]:
#step3.2.4 查看 pipeline stages, Pipeline.getStages()
dt_pipeline.getStages()

[VectorAssembler_4215b6b5f5c705c2f7ed,
 DecisionTreeClassifier_447dbb35e84fdbf76bb7]

In [26]:
#step4. 使用Pipeline 'df_pipeline' 進行資料處理與訓練
#            Pipeline 'df_pipeline' 是一個 Estimator, 所以我們使用Pipeline.fit(DataFrame)
#            以train_df 當參數,作訓練
#           訓練所得的結果是一個PipelineModel物件 (pyspark.ml.pipeline.PipelineModel) 'pipelineModel' (Transformer 物件)
#           pipelineModel 的第2個 stages 是決策樹分類器模型(DecisionTreeClassificationModel)

In [27]:
#step4.1 使用Pipeline.fit() 進行訓練
pipelineModel=dt_pipeline.fit(train_df)

In [28]:
type(pipelineModel)

pyspark.ml.pipeline.PipelineModel

In [29]:
#step4.2 查看訓練完後的決策樹模型
pipelineModel.stages[1]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_447dbb35e84fdbf76bb7) of depth 5 with 63 nodes

In [30]:
#step4.3 查看訓練完後的決策樹模型規則,DecisionTreeClassificationModel.toDebugString 屬性
print pipelineModel.stages[1].toDebugString[:500]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_447dbb35e84fdbf76bb7) of depth 5 with 63 nodes
  If (feature 0 <= 3057.0)
   If (feature 0 <= 2582.0)
    If (feature 10 <= 0.0)
     If (feature 0 <= 2405.0)
      If (feature 3 <= 0.0)
       Predict: 3.0
      Else (feature 3 > 0.0)
       Predict: 2.0
     Else (feature 0 > 2405.0)
      If (feature 17 <= 0.0)
       Predict: 2.0
      Else (feature 17 > 0.0)
       Predict: 2.0
    Else (feature 10 > 0.0)
     If (feature 9 <= 4929


In [31]:
#step5. 使用pipelineModel 進行預測

In [32]:
#step5.1 使用pipelineModel.transform(DataFrame), 參數test_df進行預測
#              預測結果是 DataFrame 物件 'predicted'
predicted = pipelineModel.transform(test_df)

In [33]:
#step5.2 檢視predicted欄位, 增加 'rawPrediction', 'probability', 'prediction' 3個欄位
print predicted .columns

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'label', 'features', 'rawPrediction', 'probability', 'prediction']


In [34]:
#step5.3 檢視預測結果的前10筆
predicted.select('features', 'rawPrediction', 'probability', 'label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|            features|       rawPrediction|         probability|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,447.0,12133....|[0.0,0.0237601658...|  5.0|       2.0|
|(54,[0,1,2,3,4,

In [37]:
#step5.4 檢視預測結果與機率
predicted.select('probability', 'prediction').take(10)

[Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0238, 0.6449, 0.0612, 0.0, 0.2701, 0.0]), prediction=2.0)]

# 21.5	評估模型的準確率

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
evaluator = MulticlassClassificationEvaluator(
                            labelCol="label", predictionCol="prediction", 
                            metricName="accuracy")

In [32]:
predictions=pipelineModel.transform(test_df)
accuracy = evaluator.evaluate(predictions)
accuracy 

0.7005240585294252

In [33]:
#21.6	使用TrainValidation進行訓練評估找出最佳模型

In [34]:
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit

In [35]:
paramGrid = ParamGridBuilder()\
  .addGrid(dt.impurity, [ "gini","entropy"])\
  .addGrid(dt.maxDepth, [ 10,15,25])\
  .addGrid(dt.maxBins, [30,40,50])\
  .build()

In [36]:
tvs = TrainValidationSplit(estimator=dt,evaluator=evaluator,
                  estimatorParamMaps=paramGrid,trainRatio=0.8)    

In [37]:
tvs_pipeline = Pipeline(stages=[vectorAssembler , tvs])

In [38]:
tvs_pipelineModel =tvs_pipeline.fit(train_df)

In [39]:
bestModel=tvs_pipelineModel.stages[1].bestModel
print bestModel.toDebugString[:500]   

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4e15848b4a4cc2cb804d) of depth 25 with 47505 nodes
  If (feature 0 <= 2717.0)
   If (feature 0 <= 2519.0)
    If (feature 0 <= 2407.0)
     If (feature 23 <= 0.0)
      If (feature 3 <= 0.0)
       If (feature 13 <= 0.0)
        If (feature 9 <= 603.0)
         If (feature 16 <= 0.0)
          Predict: 5.0
         Else (feature 16 > 0.0)
          If (feature 5 <= 379.0)
           Predict: 5.0
          Else (feature 5 > 379.0)
      


In [40]:
predictions = tvs_pipelineModel.transform(test_df)
result=predictions.withColumnRenamed("f0", "海拔") \
                                           .withColumnRenamed("f1", "方位") \
                                           .withColumnRenamed("f2", "斜率") \
                                           .withColumnRenamed("f3", "垂直距離") \
                                           .withColumnRenamed("f4", "水平距離") \
                                           .withColumnRenamed("f5", "陰影")           
result.select("海拔","方位","斜率","垂直距離" , "水平距離","陰影","label","prediction").show(10)

+------+----+----+-----+----+-----+-----+----------+
|    海拔|  方位|  斜率| 垂直距離|水平距離|   陰影|label|prediction|
+------+----+----+-----+----+-----+-----+----------+
|1863.0|37.0|17.0|120.0|18.0| 90.0|  5.0|       5.0|
|1866.0|23.0|14.0| 85.0|16.0|108.0|  2.0|       2.0|
|1871.0|22.0|22.0| 60.0|12.0| 85.0|  5.0|       5.0|
|1871.0|36.0|19.0|134.0|26.0|120.0|  5.0|       5.0|
|1874.0|18.0|14.0|  0.0| 0.0| 90.0|  5.0|       2.0|
|1877.0|19.0|18.0| 85.0|25.0|108.0|  2.0|       2.0|
|1879.0|18.0|14.0|  0.0| 0.0|120.0|  5.0|       2.0|
|1880.0|13.0|23.0| 90.0|29.0| 67.0|  5.0|       5.0|
|1883.0|27.0|24.0|120.0|24.0|108.0|  5.0|       5.0|
|1883.0|29.0|24.0| 60.0|24.0|108.0|  5.0|       5.0|
+------+----+----+-----+----+-----+-----+----------+
only showing top 10 rows



In [41]:
accuracy = evaluator.evaluate(predictions)
accuracy 

0.9283691115086464